### Use rio-viz in Jupyter Notebook

In [5]:
import time
import httpx

from folium import Map, TileLayer

from rio_viz.app import Client

In [6]:
# Create rio-viz Client (using server-thread to launch backgroud task)
client = Client("https://data.geo.admin.ch/ch.swisstopo.swissalti3d/swissalti3d_2019_2573-1085/swissalti3d_2019_2573-1085_0.5_2056_5728.tif")

# Gives some time for the server to setup
time.sleep(1)

# Check that client is running
print("Client is alive: ", client.server.is_alive())

Client is alive:  True


In [7]:
# Fetch info
httpx.get(f"{client.endpoint}/info").json()

{'bounds': [2573000.0, 1085000.0, 2574000.0, 1086000.0],
 'crs': 'http://www.opengis.net/def/crs/EPSG/0/2056',
 'band_metadata': [['b1',
   {'STATISTICS_COVARIANCES': '10685.98787505646',
    'STATISTICS_EXCLUDEDVALUES': '-9999',
    'STATISTICS_MAXIMUM': '2015.0944824219',
    'STATISTICS_MEAN': '1754.471184271',
    'STATISTICS_MINIMUM': '1615.8128662109',
    'STATISTICS_SKIPFACTORX': '1',
    'STATISTICS_SKIPFACTORY': '1',
    'STATISTICS_STDDEV': '103.37305197708'}]],
 'band_descriptions': [['b1', '']],
 'dtype': 'float32',
 'nodata_type': 'Nodata',
 'colorinterp': ['gray'],
 'scales': [1.0],
 'offsets': [0.0],
 'driver': 'GTiff',
 'count': 1,
 'width': 2000,
 'height': 2000,
 'overviews': [2, 4, 8],
 'nodata_value': -9999.0}

In [8]:
r = httpx.get(
    f"{client.endpoint}/tilejson.json",
    params = {
        "rescale": "1600,2000",  # from the info endpoint
        "colormap_name": "terrain",
    }
).json()

bounds = r["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=r["minzoom"]
)

aod_layer = TileLayer(
    tiles=r["tiles"][0],
    opacity=1,
    attr="Yo!!"
)
aod_layer.add_to(m)
m

In [9]:
client.shutdown()